In [133]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
from nltk.stem import WordNetLemmatizer
import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
import os
from glob import glob
import numpy as np
import gensim
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models as gm
from gensim import corpora, models


/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
finance = pd.read_csv("data/financial_services_rules.csv")

In [3]:
finance.head()

,title,type,abstract,document_number,html_url,pdf_url,public_inspection_pdf_url,publication_date,agencies,excerpts
0,Risk-Based Capital Guidelines; Capital Adequac...,Rule,This document corrects the Federal Reserve's r...,01-31887,https://www.federalregister.gov/documents/2001...,https://www.govinfo.gov/content/pkg/FR-2001-12...,NaN,2001-12-28,Federal Reserve System,This document corrects the Federal Reserve's r...
1,Defining the Term “Qualified Purchaser” Under ...,Proposed Rule,The Securities and Exchange Commission today p...,01-31742,https://www.federalregister.gov/documents/2001...,https://www.govinfo.gov/content/pkg/FR-2001-12...,NaN,2001-12-27,Securities and Exchange Commission,The Securities and Exchange Commission today p...
2,Home Mortgage Disclosure,Rule,The Board is publishing a final rule amending ...,01-31563,https://www.federalregister.gov/documents/2001...,https://www.govinfo.gov/content/pkg/FR-2001-12...,NaN,2001-12-26,Federal Reserve System,The Board is publishing a final rule amending ...
3,Extensions of Credit by Federal Reserve Banks;...,Rule,The Board of Governors has amended its Regulat...,01-31433,https://www.federalregister.gov/documents/2001...,https://www.govinfo.gov/content/pkg/FR-2001-12...,NaN,2001-12-21,Federal Reserve System,The Board of Governors has amended its Regulat...
4,Truth in Lending,Rule,The Board is adopting amendments to the provis...,01-31264,https://www.federalregister.gov/documents/2001...,https://www.govinfo.gov/content/pkg/FR-2001-12...,NaN,2001-12-20,Federal Reserve System,The Board is adopting amendments to the provis...


In [ ]:
from gensim.parsing.preprocessing import *
lemmatizer = WordNetLemmatizer()
def get_key_words(file_name):
    infile = open(file_name, encoding="utf8")
    contents = infile.read()
    CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_multiple_whitespaces,
                   strip_numeric, remove_stopwords, strip_short, strip_non_alphanum]
    doc = preprocess_string(contents, CUSTOM_FILTERS)
    doc = list(map(lambda x: lemmatizer.lemmatize(x),doc))
    infile.close()
    return list(set(doc))

# Pre 2007

In [4]:
finance['publication_date'].min()

'2001-01-03'

In [10]:
pre_2007  = finance[finance['publication_date'] < '2007-01-01']

In [7]:
headers = { 'Accept-Language' : "en-US,en;q=0.9,hi;q=0.8,ur;q=0.7,bn;q=0.6",
            'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}


In [11]:
pre_2007.columns

Index(['title', 'type', 'abstract', 'document_number', 'html_url', 'pdf_url',
       'public_inspection_pdf_url', 'publication_date', 'agencies',
       'excerpts'],
      dtype='object')

In [135]:
# rems = [summ[30].text,summ[34].text,summ[35].text, summ[36].text, summ[42].text, summ[209].text]
rems = np.load("remove_pars.npy",allow_pickle=True)

In [105]:
# np.save("remove_pars.npy",rems)

In [ ]:
# for i in range(len(summ)):
#     if len(summ[i].text) > 300:
#         print(i)

In [186]:
for j in range(len(pre_2007)):
    content = requests.get(pre_2007.iloc[j]['html_url'],headers=headers).text
    soup = BeautifulSoup(content, 'html.parser')
    
    summ = soup.find_all('p')
    print(len(summ))
    with open(f"data/pre_2007/{pre_2007.iloc[j]['document_number']}.xml","w") as f:
        for i in summ:
            if len(i.text)>300:
                if (i.text == rems[0]) or (i.text == rems[1])or (i.text == rems[2])or (i.text == rems[3]) or (i.text == rems[4]):
                    continue
                else:
                    f.write(i.text)
    print(f"Page: {j} completed")


527
Page: 461 completed
172
Page: 462 completed
369
Page: 463 completed
1743
Page: 464 completed
65
Page: 465 completed
214
Page: 466 completed
69
Page: 467 completed
396
Page: 468 completed
886
Page: 469 completed
2096
Page: 470 completed
618
Page: 471 completed
181
Page: 472 completed
226
Page: 473 completed
239
Page: 474 completed
69
Page: 475 completed
119
Page: 476 completed
206
Page: 477 completed
174
Page: 478 completed
69
Page: 479 completed
161
Page: 480 completed
1999
Page: 481 completed
48
Page: 482 completed
79
Page: 483 completed
85
Page: 484 completed
195
Page: 485 completed
101
Page: 486 completed
237
Page: 487 completed
53
Page: 488 completed
80
Page: 489 completed
121
Page: 490 completed
77
Page: 491 completed
388
Page: 492 completed
199
Page: 493 completed
65
Page: 494 completed
73
Page: 495 completed
356
Page: 496 completed
69
Page: 497 completed
179
Page: 498 completed
94
Page: 499 completed
62
Page: 500 completed
230
Page: 501 completed
69
Page: 502 completed
293
P

In [142]:
# g = get_key_words("/Users/atufashireen/projects/Tamu_Bloomberg/01-31887.xml")
base_dir = "/Users/atufashireen/projects/Tamu_Bloomberg/data/pre_2007"

In [188]:
processed_docs = []
for file in glob.glob(f"{base_dir}/*"):
    processed_docs.append(get_key_words(file))

In [196]:
dictionary = gensim.corpora.Dictionary(processed_docs)
# dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n= 100000)
corpus = [dictionary.doc2bow(text) for text in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [207]:
num_topics = 5
lda_model = LdaModel(corpus_tfidf, num_topics=num_topics)
vis = gm.prepare(topic_model=lda_model, corpus=corpus_tfidf, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)


/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [209]:
lda_model.save("models/pre_2007_model")

In [225]:
np.save("pre_2007_corpus",processed_docs)


/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


* topic 1 talks about the Gramm-Leach-Bliley Act
* topic 2 talks about corrections in the occ regulations.
* topic 3 talks about imporatnace of fostering of sustainability.


# Obama Admin (2009-2017)

In [246]:
pre_2007  = finance[(finance['publication_date'] > '2008-12-31') & (finance['publication_date'] < '2018-01-01')]

In [247]:
pre_2007['publication_date'].min()

'2009-01-14'

In [248]:
pre_2007.head()

,title,type,abstract,document_number,html_url,pdf_url,public_inspection_pdf_url,publication_date,agencies,excerpts
1177,Reserve Requirements of Depository Institution...,Proposed Rule,The Board is requesting public comment on prop...,E9-31040,https://www.federalregister.gov/documents/2009...,https://www.govinfo.gov/content/pkg/FR-2009-12...,NaN,2009-12-31,Federal Reserve System,The Board is requesting public comment on prop...
1178,Revised Adjusted Net Capital Requirements for ...,Rule,The Commodity Futures Trading Commission (``Co...,E9-31058,https://www.federalregister.gov/documents/2009...,https://www.govinfo.gov/content/pkg/FR-2009-12...,NaN,2009-12-31,Commodity Futures Trading Commission,The Commodity Futures Trading Commission (``Co...
1179,Electronic Filing of Financial Reports and Not...,Rule,The Commodity Futures Trading Commission (``Co...,E9-31032,https://www.federalregister.gov/documents/2009...,https://www.govinfo.gov/content/pkg/FR-2009-12...,NaN,2009-12-30,Commodity Futures Trading Commission,The Commodity Futures Trading Commission (``Co...
1180,Temporary Rule Regarding Principal Trades With...,Rule,The Securities and Exchange Commission is adop...,E9-30877,https://www.federalregister.gov/documents/2009...,https://www.govinfo.gov/content/pkg/FR-2009-12...,NaN,2009-12-30,Securities and Exchange Commission,The Securities and Exchange Commission is adop...
1181,Community Reinvestment Act Regulations,Rule,"The OCC, the Board, the FDIC, and the OTS (col...",E9-30646,https://www.federalregister.gov/documents/2009...,https://www.govinfo.gov/content/pkg/FR-2009-12...,NaN,2009-12-29,"Treasury Department, Comptroller of the Curren...","The OCC, the Board, the FDIC, and the OTS (col..."


In [221]:
pre_2007.shape

(1121, 10)

In [219]:
headers = { 'Accept-Language' : "en-US,en;q=0.9,hi;q=0.8,ur;q=0.7,bn;q=0.6",
            'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}


In [222]:
rems = np.load("remove_pars.npy",allow_pickle=True)

In [ ]:
for j in range(len(pre_2007)):
    content = requests.get(pre_2007.iloc[j]['html_url'],headers=headers).text
    soup = BeautifulSoup(content, 'html.parser')
    
    summ = soup.find_all('p')
    print(len(summ))
    with open(f"data/obama_admin/{pre_2007.iloc[j]['document_number']}.xml","w") as f:
        for i in summ:
            if len(i.text)>300:
                if (i.text == rems[0]) or (i.text == rems[1])or (i.text == rems[2])or (i.text == rems[3]) or (i.text == rems[4]):
                    continue
                else:
                    f.write(i.text)
    print(f"Page: {j} completed")

In [233]:
# g = get_key_words("/Users/atufashireen/projects/Tamu_Bloomberg/01-31887.xml")
base_dir = "/Users/atufashireen/projects/Tamu_Bloomberg/data/obama_admin"

In [234]:
processed_docs = []
for file in glob.glob(f"{base_dir}/*"):
    processed_docs.append(get_key_words(file))

In [235]:
dictionary = gensim.corpora.Dictionary(processed_docs)
# dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n= 100000)
corpus = [dictionary.doc2bow(text) for text in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [236]:
num_topics = 5
lda_model = LdaModel(corpus_tfidf, num_topics=num_topics,id2word=dictionary)
vis = gm.prepare(topic_model=lda_model, corpus=corpus_tfidf, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)


/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation

In [237]:
np.save("data/obama_admin/obama_admin.npy",processed_docs)
lda_model.save("models/models_obama_admin/obama_admin_model")

/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


* Topic 1 talks about corrections or creating editions to APA or Advanced Pricing Agreements.
* Topic 2 talks about opprtunities for lafayetti City.
* Topic 3 talks about CLA (consumer leasing agreement) and CPI (consumer Price Index).
* The further topics has leasing ,mortagage which is indicates the house crisis of 2008.

# Trump Admin (2017-2021)

In [252]:
pre_2007  = finance[(finance['publication_date'] > '2016-12-31') & (finance['publication_date'] < '2022-01-01')]
print(pre_2007['publication_date'].min())
pre_2007.head()

2017-01-10


,title,type,abstract,document_number,html_url,pdf_url,public_inspection_pdf_url,publication_date,agencies,excerpts
2702,Restrictions on Qualified Financial Contracts ...,Rule,This document makes technical corrections to r...,2017-27971,https://www.federalregister.gov/documents/2017...,https://www.govinfo.gov/content/pkg/FR-2017-12...,https://public-inspection.federalregister.gov/...,2017-12-28,Federal Deposit Insurance Corporation,This document makes technical corrections to r...
2703,Community Reinvestment Act Regulations,Rule,"The OCC, the Board, and the FDIC (collectively...",2017-27813,https://www.federalregister.gov/documents/2017...,https://www.govinfo.gov/content/pkg/FR-2017-12...,https://public-inspection.federalregister.gov/...,2017-12-27,"Treasury Department, Comptroller of the Curren...","The OCC, the Board, and the FDIC (collectively..."
2704,Home Mortgage Disclosure (Regulation C) Adjust...,Rule,The Bureau of Consumer Financial Protection (B...,2017-27879,https://www.federalregister.gov/documents/2017...,https://www.govinfo.gov/content/pkg/FR-2017-12...,https://public-inspection.federalregister.gov/...,2017-12-27,Consumer Financial Protection Bureau,The Bureau of Consumer Financial Protection (B...
2705,Truth in Lending Act (Regulation Z) Adjustment...,Rule,The Bureau is amending the official commentary...,2017-27897,https://www.federalregister.gov/documents/2017...,https://www.govinfo.gov/content/pkg/FR-2017-12...,https://public-inspection.federalregister.gov/...,2017-12-27,Consumer Financial Protection Bureau,The Bureau is amending the official commentary...
2706,Home Mortgage Disclosure,Rule,The Board of Governors of the Federal Reserve ...,2017-27491,https://www.federalregister.gov/documents/2017...,https://www.govinfo.gov/content/pkg/FR-2017-12...,https://public-inspection.federalregister.gov/...,2017-12-22,Federal Reserve System,The Board of Governors of the Federal Reserve ...


In [253]:
pre_2007.shape

(928, 10)

In [254]:
for j in range(len(pre_2007)):
    content = requests.get(pre_2007.iloc[j]['html_url'],headers=headers).text
    soup = BeautifulSoup(content, 'html.parser')
    
    summ = soup.find_all('p')
    print(len(summ))
    with open(f"data/trump_admin/{pre_2007.iloc[j]['document_number']}.xml","w") as f:
        for i in summ:
            if len(i.text)>300:
                if (i.text == rems[0]) or (i.text == rems[1])or (i.text == rems[2])or (i.text == rems[3]) or (i.text == rems[4]):
                    continue
                else:
                    f.write(i.text)
    print(f"Page: {j} completed")

67
Page: 0 completed
102
Page: 1 completed
80
Page: 2 completed
97
Page: 3 completed
77
Page: 4 completed
72
Page: 5 completed
81
Page: 6 completed
181
Page: 7 completed
143
Page: 8 completed
236
Page: 9 completed
151
Page: 10 completed
72
Page: 11 completed
184
Page: 12 completed
200
Page: 13 completed
93
Page: 14 completed
749
Page: 15 completed
70
Page: 16 completed
242
Page: 17 completed
63
Page: 18 completed
72
Page: 19 completed
70
Page: 20 completed
63
Page: 21 completed
188
Page: 22 completed
5053
Page: 23 completed
80
Page: 24 completed
106
Page: 25 completed
95
Page: 26 completed
92
Page: 27 completed
83
Page: 28 completed
1329
Page: 29 completed
109
Page: 30 completed
234
Page: 31 completed
784
Page: 32 completed
111
Page: 33 completed
1255
Page: 34 completed
72
Page: 35 completed
126
Page: 36 completed
417
Page: 37 completed
152
Page: 38 completed
120
Page: 39 completed
63
Page: 40 completed
90
Page: 41 completed
70
Page: 42 completed
276
Page: 43 completed
112
Page: 44 com

In [255]:
base_dir = "/Users/atufashireen/projects/Tamu_Bloomberg/data/trump_admin"
processed_docs = []
for file in glob.glob(f"{base_dir}/*"):
    processed_docs.append(get_key_words(file))

In [261]:
dictionary = gensim.corpora.Dictionary(processed_docs)
# dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n= 100000)
corpus = [dictionary.doc2bow(text) for text in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [262]:
num_topics = 5
lda_model = LdaModel(corpus_tfidf, num_topics=num_topics,id2word=dictionary)
vis = gm.prepare(topic_model=lda_model, corpus=corpus_tfidf, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)


/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation

In [267]:
lda_model.save("models/models_trump_admin/trump_admin")
np.save("data/trump_admin/trump_admin.npy",processed_docs)

/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# Covid (01-01-2020 ~)

In [264]:
pre_2007  = finance[(finance['publication_date'] > '2019-12-31')]
print(pre_2007['publication_date'].min())
pre_2007.head()

2020-01-08


,title,type,abstract,document_number,html_url,pdf_url,public_inspection_pdf_url,publication_date,agencies,excerpts
3245,Notification of Inflation Adjustments for Civi...,Rule,The Office of the Comptroller of the Currency ...,2020-28942,https://www.federalregister.gov/documents/2020...,https://www.govinfo.gov/content/pkg/FR-2020-12...,https://public-inspection.federalregister.gov/...,2020-12-31,"Treasury Department, Comptroller of the Currency",The Office of the Comptroller of the Currency ...
3246,Regulation ATS for ATSs That Trade U.S. Govern...,Proposed Rule,The Securities and Exchange Commission is prop...,2020-21781,https://www.federalregister.gov/documents/2020...,https://www.govinfo.gov/content/pkg/FR-2020-12...,https://public-inspection.federalregister.gov/...,2020-12-31,Securities and Exchange Commission,The Securities and Exchange Commission is prop...
3247,Amendments to the Commission's Rules of Practice,Rule,The Securities and Exchange Commission (``Comm...,2020-25747,https://www.federalregister.gov/documents/2020...,https://www.govinfo.gov/content/pkg/FR-2020-12...,https://public-inspection.federalregister.gov/...,2020-12-30,Securities and Exchange Commission,The Securities and Exchange Commission (``Comm...
3248,Delegation of Authority to Director of the Div...,Rule,The Securities and Exchange Commission (``Comm...,2020-27537,https://www.federalregister.gov/documents/2020...,https://www.govinfo.gov/content/pkg/FR-2020-12...,https://public-inspection.federalregister.gov/...,2020-12-29,Securities and Exchange Commission,The Securities and Exchange Commission (``Comm...
3249,Qualified Mortgage Definition Under the Truth ...,Rule,"With certain exceptions, Regulation Z requires...",2020-27567,https://www.federalregister.gov/documents/2020...,https://www.govinfo.gov/content/pkg/FR-2020-12...,https://public-inspection.federalregister.gov/...,2020-12-29,Consumer Financial Protection Bureau,"With certain exceptions, Regulation Z requires..."


In [265]:
pre_2007.shape

(385, 10)

In [270]:
for j in range(117, len(pre_2007)):
    content = requests.get(pre_2007.iloc[j]['html_url'],headers=headers).text
    soup = BeautifulSoup(content, 'html.parser')
    
    summ = soup.find_all('p')
    print(len(summ))
    with open(f"data/covid/{pre_2007.iloc[j]['document_number']}.xml","w") as f:
        for i in summ:
            if len(i.text)>300:
                if (i.text == rems[0]) or (i.text == rems[1])or (i.text == rems[2])or (i.text == rems[3]) or (i.text == rems[4]):
                    continue
                else:
                    f.write(i.text)
        try:
            f.write(pre_2007.iloc[j]['title'])
            f.write(pre_2007.iloc[j]['abstract'])
        except Exception as e:
            print("Errororr",e)
    print(f"Page: {j} completed")

337
Page: 117 completed
247
Page: 118 completed
264
Page: 119 completed
148
Page: 120 completed
295
Page: 121 completed
1001
Page: 122 completed
91
Page: 123 completed
121
Page: 124 completed
166
Page: 125 completed
449
Page: 126 completed
844
Page: 127 completed
74
Page: 128 completed
107
Page: 129 completed
235
Page: 130 completed
147
Page: 131 completed
315
Page: 132 completed
156
Page: 133 completed
822
Page: 134 completed
309
Page: 135 completed
711
Page: 136 completed
112
Page: 137 completed
595
Page: 138 completed
169
Page: 139 completed
63
Page: 140 completed
568
Page: 141 completed
219
Page: 142 completed
82
Page: 143 completed
266
Page: 144 completed
89
Page: 145 completed
126
Page: 146 completed
300
Page: 147 completed
98
Page: 148 completed
109
Page: 149 completed
682
Page: 150 completed
84
Page: 151 completed
294
Page: 152 completed
1886
Page: 153 completed
90
Page: 154 completed
2247
Page: 155 completed
516
Page: 156 completed
177
Page: 157 completed
309
Page: 158 complet

In [271]:
base_dir = "/Users/atufashireen/projects/Tamu_Bloomberg/data/covid"
processed_docs = []
for file in glob.glob(f"{base_dir}/*"):
    processed_docs.append(get_key_words(file))

In [274]:
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n= 100000)
corpus = [dictionary.doc2bow(text) for text in processed_docs]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [279]:
num_topics = 5
lda_model = LdaModel(corpus_tfidf, num_topics=num_topics,id2word=dictionary)
vis = gm.prepare(topic_model=lda_model, corpus=corpus_tfidf, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)


/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [281]:
lda_model.save("models/models_covid/covid")
np.save("data/covid/covid.npy",processed_docs)

/Users/atufashireen/projects/Tamu_Bloomberg/venv/lib/python3.9/site-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


# Removing clutter

In [286]:
del processed_docs, pre_2007

In [ ]:
glob.glob("/Users/atufashireen/projects/Tamu_Bloomberg/data/covid/*")

In [285]:
covid = "/Users/atufashireen/projects/Tamu_Bloomberg/data/covid/*"
for i in glob.glob(covid):
    if i.split('/')[-1].split('.')[-1]=='npy':
        continue
    else:
        os.remove(i)

In [287]:
pre_2007 = "/Users/atufashireen/projects/Tamu_Bloomberg/data/pre_2007/*"
for i in glob.glob(pre_2007):
    if i.split('/')[-1].split('.')[-1]=='npy':
        continue
    else:
        os.remove(i)

In [288]:
trump_admin = "/Users/atufashireen/projects/Tamu_Bloomberg/data/trump_admin/*"
for i in glob.glob(trump_admin):
    if i.split('/')[-1].split('.')[-1]=='npy':
        continue
    else:
        os.remove(i)

In [ ]:
obama_admin = "/Users/atufashireen/projects/Tamu_Bloomberg/data/obama_admin/*"
for i in glob.glob(obama_admin):
    if i.split('/')[-1].split('.')[-1]=='npy':
        continue
    else:
        os.remove(i)